In [27]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [28]:
from encode_data import *
from midi_data import *

In [29]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel, transform_csv_row

In [30]:
from collections import Counter

In [31]:
import scipy.sparse

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [32]:
version = 'v6'
data_path = Path('data/midi')
version_path = data_path/version

In [33]:
source_dir = 'midi_transform'
out_dir = 'midi_npz'
source_csv = version_path/source_dir/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'

In [8]:
df = pd.read_csv(out_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,9,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,genres,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform,midi_npz,midi_npz_timesteps
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,NaN,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,NaN,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,Jazz,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...,midi_npz/hooktheory/pianoroll/w/whiteflame/sen...,129.0
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,Holiday,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...,129.0


In [9]:
df_filtered = df.loc[df[out_dir].notna()]; df_filtered.shape

(38557, 25)

In [11]:
outliers = []
for idx,row in df_filtered.iterrows():
    np_file = row[out_dir]
    chordarr = load_chordarr(Path(version_path)/np_file)
    if np.any(chordarr > 100): outliers.append(np_file)
    if len(outliers) > 10: break

In [12]:
outliers

['midi_npz/hooktheory/pianoroll/w/war/low-rider/intro-and-verse_key.npz',
 'midi_npz/hooktheory/pianoroll/y/ylvis/massachusetts/intro_key.npz',
 'midi_npz/hooktheory/pianoroll/y/yo-la-tengo/ohm/verse_key.npz',
 'midi_npz/hooktheory/pianoroll/j/jose-gonzales/step-out/intro_key.npz',
 'midi_npz/hooktheory/pianoroll/j/john-williams/midway-march/verse_key.npz',
 'midi_npz/hooktheory/pianoroll/j/jon-foreman/white-as-snow/intro_key.npz',
 'midi_npz/hooktheory/pianoroll/j/judas-priest/breaking-the-law/chorus_key.npz',
 'midi_npz/hooktheory/pianoroll/j/jocelyn-pook/masked-ball---eyes-wide-shut/chorus_key.npz',
 'midi_npz/hooktheory/pianoroll/j/jay-alphonso-williams/john-cena---the-time-is-now/pre-chorus-and-chorus_key.npz',
 'midi_npz/hooktheory/pianoroll/g/game-freak/vs-brendan-and-may/chorus_key.npz',
 'midi_npz/hooktheory/pianoroll/g/gizmondo-studios/sticky-balls-theme-song/verse_key.npz']

In [43]:
max_index = 0
for idx,row in df_filtered.iterrows():
    np_file = row[out_dir]
    chordarr = load_chordarr(Path(version_path)/np_file)
    m = chordarr.sum(axis=1).argmax(axis=-1).max()
    if m > max_index: 
        max_index = m
        print(m)

70
76
81
84
88
93
100
108
112
124
125
126


In [ ]:
m

In [13]:
f = 'midi_npz/hooktheory/pianoroll/g/gizmondo-studios/sticky-balls-theme-song/verse_key.npz'
a = load_chordarr(Path(version_path/f))

In [34]:
a.shape

(129, 3, 127)

In [40]:
a.sum(axis=1).argmax(axis=-1).max()

76

In [26]:
np.all((a != -2), axis=(1,2))

array([ True, False, False, False, ..., False, False, False,  True])

In [18]:
def calc_timesteps(idxrow):
    idx,row = idxrow
    np_file = row[out_dir]
    if not isinstance(np_file, str) or not (Path(version_path)/np_file).exists(): return idx,None
    try:
#         timesteps = scipy.sparse.load_npz(np_file).shape[0]
        timesteps = load_chordarr(Path(version_path)/np_file).shape[0]
        return idx,timesteps
    except Exception as e:
        print('Error reading text', e, np_file)
    return idx, None

In [19]:
# for r in merged_df.iterrows():
#     calc_timesteps(r)